In [1]:
import sys
import os
import openai
from openai import OpenAI

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file


openai.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

In [33]:
import time
def upload_file(local_path):
    file=client.files.create(
        file=open(local_path, 'rb'),
        purpose='assistants'
    )
    return file

def create_assistant(file):
    assistant = client.beta.assistants.create(
        name="Test assistant",
        instructions="Test instructions",
        tools=[{"type":"retrieval"}],
        model="gpt-4-1106-preview",
        file_ids=[file.id],
    )
    return assistant

def generate_response(message_body):
    thread = client.beta.threads.create()
    thread_id = thread.id
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )
    return thread_id
    
def run_assistant(assistant_id, thread_id):
    assistant = client.beta.assistants.retrieve(assistant_id)
    thread = client.beta.threads.retrieve(thread_id)
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    
    #wait for completion of the run
    while run.status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    # logging.info(f"Generated Messages: {new_message}" )
    return new_message

In [31]:
print(run_assistant("asst_nb7ScJrV5TmrRdSEY2AI5wve"))

The search did not return relevant information about the winner of the 1986 World Series. Given the nature of the document, it appears to be a driver's manual for Connecticut, which would not likely contain information about the World Series. 

To answer your question directly, the New York Mets won the 1986 World Series, defeating the Boston Red Sox in seven games.


In [10]:
file = upload_file("CT_Drivers_Manual.pdf")

In [11]:
print (file)

FileObject(id='file-QaneotyETXdU3P2qmhtfBdbd', bytes=4661101, created_at=1701228302, filename='CT_Drivers_Manual.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [12]:
assistant = create_assistant(file)

In [36]:
thread_id = generate_response("What does the manual say about bicyclists?")
print(run_assistant("asst_nb7ScJrV5TmrRdSEY2AI5wve", thread_id))


The manual states that when used on roadways, bicyclists are considered vehicles and are expected to follow the same rules of the road as motorized vehicles. Motorists should be aware that a bicyclist has the same rights, privileges, and responsibilities as them.

Here are the key points mentioned about bicyclists in the manual:

- **Bicyclists on the Road:** They should be expected on the road, obeying traffic rules and potentially using the left lane when turning. If a lane is too narrow for cars and bikes to drive side-by-side, a bicyclist may take the travel lane, meaning occupying the center of the lane.

- **Visibility and Skill Levels:** Bicyclists may not be easily seen. Motorists must be careful when approaching bicyclists, understanding that they come with varying levels of skill.

**For Bicyclists:**

- They must ride on the right side of the road as far to the right as safely possible. They are allowed on the left edge of a right turn lane if going straight.
- They must obe